## Libraires

In [2]:
# !pip install nltk

In [3]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import string
from sklearn.metrics import accuracy_score
# Download NLTK stopwords (only once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag, word_tokenize

# If not done already, download necessary NLTK resources
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

import random
import warnings
warnings.filterwarnings("ignore")
nltk.download('wordnet')
from nltk.corpus import wordnet
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from transformers import BertForSequenceClassification, get_scheduler
from torch.optim import AdamW
from tqdm import tqdm
import matplotlib.pyplot as plt
import os

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Mounting Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Loading Dataset

In [5]:
dataset_path='/content/drive/MyDrive/techNL_HSTIP'
df = pd.read_excel(dataset_path+'/Cleaned_HS_Tech_Internship_Data.xlsx')
df

,first_name,last_name,town,identify_as,specify_below,why_internship,tech_experience,non_tech_experience,goals,other_comments,selected
0,Fuad,Ahmed,ST. JOHN'S,NaN,NaN,I am applying for this internship because I be...,I do not have any experience with any technolo...,Quick Learner\nTechnologically competent\nHard...,My plans are to pursue a degree in engineering...,NaN,no
1,Lucas,Maloney,Avondale,NaN,NaN,I am applying for an internship because I am v...,The only experience I have is using computers ...,"I am a hard worker, organized, video gamer, mu...",After graduation in June I will work for the s...,NaN,no
2,Damon,Pitts,Mount Pearl,Male,NaN,I am applying for this internship for several ...,I have a variety of experience working with te...,Computer Software / Technical Skills\nâ€¢\tMic...,"After high school graduation, I am considering...",NaN,no
3,Riley,Hiron,Corner Brook,Male,NaN,I am applying for this internship in the hope'...,I have some home projects such as making some ...,I know how to program in python and c# and I ...,Post secondary at collage of the north Atlanti...,NaN,no
4,Keenan,Crawley,Avondale,Male,NaN,This job opportunity caught my eye. The reason...,"I've built my own computer from scratch, and h...",Google suite for education. Along with the use...,A goal I would want to see fulfilled and in my...,NaN,no
...,...,...,...,...,...,...,...,...,...,...,...
1832,Chloe,Wilson,Torbay,Female,NaN,"""I’m eager to learn more about technology and ...","""Have learned basic HTML, CSS, and JavaScript ...","""Volunteered for a community event, learning e...","""I’m considering a career in tech or marketing...",NaN,yes
1833,Mason,Clark,Gander,Male,NaN,"""I’m curious about the tech field and unsure i...","""I’ve worked on a small web development projec...","""I’ve volunteered with youth groups, learning ...","""I’m thinking about tech, but I’m also conside...",NaN,no
1834,Ella,Mitchell,Clarenville,Female,NaN,"""I’m applying to see if tech could be the righ...","""Created a personal website using WordPress an...","""Worked as a volunteer coordinator for a non-p...","""Exploring tech but also considering public re...",NaN,no
1835,Ben,Allen,St. John's,Male,NaN,"""I’m interested in tech but haven’t fully comm...","""Created a portfolio website using HTML, CSS, ...","""Organized local tech meetups for university s...","""Tech exploration with an interest in entrepre...",NaN,no


In [6]:
df['identify_as'].unique()

array([nan, 'Male', 'Female', 'Non-binary/2-Spirit/Agender/Genderfluid',
       'Male;Female;Non-binary/2-Spirit/Agender/Genderfluid',
       'Nonbinary as of now, still questioning somewhat',
       'Prefer not to say', 'Woman, Racialized minority', 'Woman',
       'Racialized minority, Newcomer to Canada',
       'Woman, 2SLGBTQ+ and/or gender diverse', 'Newcomer to Canada',
       'Racialized minority', 'Newcomer to Canada, Woman',
       'Aboriginal and/or indigenous', 'Please specify below',
       'Aboriginal and/or indigenous, 2SLGBTQ+ and/or gender diverse',
       '2SLGBTQ+ and/or gender diverse',
       '2SLGBTQ+ and/or gender diverse, Woman',
       'Woman, Racialized minority, Newcomer to Canada',
       'Person with disability',
       '2SLGBTQ+ and/or gender diverse, Person with disability',
       'Woman, Newcomer to Canada',
       'Woman, Person with disability, 2SLGBTQ+ and/or gender diverse',
       '2SLGBTQ+ and/or gender diverse, Woman, Person with disability',
   

In [7]:
diversity_keywords = {
    'woman': 1,
    'female': 1,
    'non binary': 1,
    'non-binary': 1,
    'genderfluid': 1,
    'agender': 1,
    '2slgbtq+': 1,
    'slgbtq': 1,
    'lgbtq': 1,
    'gender diverse': 1,
    'racialized minority': 1,
    'person disability': 1,
    'person with disability': 1,
    'disability': 1,
    'aboriginal': 1,
    'indigenous': 1,
    'newcomer canada': 1,
    'francophone': 1,
}

max_score = len(set(diversity_keywords))  # count of unique diversity indicators

def calculate_diversity_score(text):
    if pd.isna(text) or text.strip() == '':
        return 0
    text = text.lower()
    score = 0
    matched = set()
    for key in diversity_keywords:
        if key in text and key not in matched:
            score += diversity_keywords[key]
            matched.add(key)
    return round(min(score / max_score, 1), 2)

df['diversity_score'] = df['identify_as'].apply(calculate_diversity_score)

print(df[['identify_as', 'diversity_score']])


     identify_as  diversity_score
0            NaN             0.00
1            NaN             0.00
2           Male             0.00
3           Male             0.00
4           Male             0.00
...          ...              ...
1832      Female             0.06
1833        Male             0.00
1834      Female             0.06
1835        Male             0.00
1836        Male             0.00

[1837 rows x 2 columns]


In [8]:
df['diversity_score'].value_counts()

,count
diversity_score,
0.00,1016
0.06,671
0.11,66
0.28,29
0.22,25
0.17,13
0.33,12
0.39,5


In [9]:
df['selected'] = df['selected'].str.strip().str.lower().map({
    'yes': 'yes',
    'no': 'no',
    'declined': 'no'
})

df['selected'].value_counts()

,count
selected,
yes,978
no,859


In [10]:
missing_counts = df.isna().sum()
print(missing_counts[missing_counts > 0])

identify_as            221
specify_below          659
why_internship           1
tech_experience         19
non_tech_experience      2
goals                  141
other_comments         326
dtype: int64


In [11]:
df.shape

(1837, 12)

In [12]:
# df = df.dropna(subset=['specify_below'])
# df = df.drop(columns=['specify_below'])
df = df.drop(columns=['identify_as'])


In [13]:
df.shape

(1837, 11)

In [14]:
# Initialize
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')
stop_words.remove('not')  # Keep 'not' for better negation handling

# POS tag converter
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Clean + lemmatize function
def clean_and_lemmatize(text):
    if pd.isnull(text):
        return ""
    text = str(text)
    text = re.sub(r'\bRT\b', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub('[^a-zA-Z\s]', ' ', text)
    text = text.lower()

    tokens = word_tokenize(text)
    tagged_tokens = pos_tag(tokens)
    lemmatized_words = [
        lemmatizer.lemmatize(word, get_wordnet_pos(tag))
        for word, tag in tagged_tokens
        if word not in stop_words
    ]
    return ' '.join(lemmatized_words)

# List of your columns to process
cols_to_clean = ['town','why_internship', 'tech_experience', 'non_tech_experience', 'goals', 'other_comments', 'diversity_score']

# Apply the cleaning function to each column
for col in cols_to_clean:
    df[col] = df[col].apply(clean_and_lemmatize)


In [15]:
df = df[800:]
df.shape

(1037, 11)

### BERT Base Model

#### Initial Steps

In [16]:
# 1. Combine text fields

df['combined_text'] = df[['town',	'diversity_score','why_internship', 'tech_experience', 'non_tech_experience', 'goals', 'other_comments']].agg(' '.join, axis=1)

# 2. Map labels
df['label'] = df['selected'].map({'yes': 1, 'no': 0})

# 3. Initial train-test split (80% train+val, 20% test)
train_val_texts, test_texts, train_val_labels, test_labels = train_test_split(
    df['combined_text'].values,
    df['label'].values,
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

# 4. Split train+val into final train (64%) and val (16%)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_val_texts,
    train_val_labels,
    test_size=0.2,
    random_state=42,
    stratify=train_val_labels
)

# 5. Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=256)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [17]:
# Dataset class
class InternSelectionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

# Create datasets
train_dataset = InternSelectionDataset(train_encodings, train_labels)
val_dataset = InternSelectionDataset(val_encodings, val_labels)
test_dataset = InternSelectionDataset(test_encodings, test_labels)


In [18]:
from transformers import BertForSequenceClassification, get_scheduler
from torch.utils.data import DataLoader
import torch

# Load BERT-base model for binary classification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,
    hidden_dropout_prob=0.12
)

# Use GPU if available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# RMSprop optimizer
optimizer = torch.optim.RMSprop(model.parameters(), lr=1e-5, weight_decay=0.01)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=64)
test_loader  = DataLoader(test_dataset, batch_size=64)
# Scheduler
num_epochs = 16
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=100,
    num_training_steps=num_training_steps
)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score

# Early stopping params
patience = 3
min_delta = 1e-4
best_val_loss = float('inf')
patience_counter = 0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", leave=False)

    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        loop.set_postfix(train_loss=loss.item())

    avg_train_loss = train_loss / len(train_loader)

    # --- Validation ---
    model.eval()
    val_loss = 0
    all_preds = []
    all_labels = []

    val_loop = tqdm(val_loader, desc=f"Epoch {epoch + 1} Validation", leave=False)
    with torch.no_grad():
        for batch in val_loop:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            val_loss += loss.item()
            preds = torch.argmax(outputs.logits, dim=1)
            labels = batch["labels"]
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            val_loop.set_postfix(val_loss=val_loss/len(all_preds), val_acc=accuracy_score(all_labels, all_preds))

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = accuracy_score(all_labels, all_preds)

    print(f"✅ Epoch {epoch + 1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.4f}")

    # Early stopping logic
    if avg_val_loss < best_val_loss - min_delta:
        best_val_loss = avg_val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"⏹️ Early stopping triggered. No improvement for {patience} epochs.")
        break


Epoch 1/16:   0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import torch

# --- Evaluate on test set ---
model.eval()
all_test_preds = []
all_test_labels = []

with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=1)
        labels = batch["labels"]
        all_test_preds.extend(preds.cpu().numpy())
        all_test_labels.extend(labels.cpu().numpy())

# --- Metrics ---
print("🔍 Classification Report (Test Set):\n")
print(classification_report(all_test_labels, all_test_preds, target_names=["Not Selected", "Selected"]))




In [ ]:
# --- Confusion Matrix ---
cm = confusion_matrix(all_test_labels, all_test_preds)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Not Selected", "Selected"], yticklabels=["Not Selected", "Selected"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("🧠 Confusion Matrix - Test Set")
plt.show()

In [ ]:
# Save only the model weights
os.chdir(dataset_path)
torch.save(model.state_dict(), "internship_selection_model.pt")
print("✅ Model weights saved as 'internship_selection_model.pt'")


In [19]:
# Re-initialize the same model architecture
os.chdir(dataset_path)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.load_state_dict(torch.load("internship_selection_model.pt", map_location=torch.device('cpu')))
model.to('cpu')
model.eval()
print("✅ Model loaded from 'internship_selection_model.pt'")


✅ Model loaded from 'internship_selection_model.pt'


In [20]:

# Define or import your model architecture
# For example, if you're using a BERT-based classifier:
from transformers import BertForSequenceClassification

# Load the correct model architecture
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Set path to the model file
model_path = "/content/drive/MyDrive/techNL_HSTIP/internship_selection_model.pt"

# Set the device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Load the model weights
model.load_state_dict(torch.load(model_path, map_location=device))

# Move model to appropriate device
model.to(device)
model.eval()

print("✅ Model loaded from 'internship_selection_model_NEW.pt'")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model loaded from 'internship_selection_model_NEW.pt'


### Testing

In [21]:


sample_1 = {
    'town': 'Corner brook',
    'identify_as': 'Female',
    "why_internship": "I'm sure it will be fun",
    "tech_experience": "low experience",
    "non_tech_experience": "Debate",
    "goals": "unsure",
    "other_comments": ""
}


In [ ]:
sample_3 = {
    'town': 'St Johns',
    # 'identify_as': 'Male',
    'diversity_score': '0.0',
    'specify_below': ' ',
    "why_internship": "I have an interest in tech and hope to have a tech profession in the future. This opportunity will help me solidify that dream.",
    "tech_experience": "i did a few programming courses at my school",
    "non_tech_experience": "I am a dedicated volunteer in my community. I come from a larger size family where team work makes the dream work where I am the second eldest so a lot of responsibilities fall onto me. I have played on many all star soccer teams and school teams all throughout my time in school and I am personable and approachable and eager to learn.",
    "goals": "I will be attending MUN in September for engineering.",
    "other_comments": "I am excited for this opportunity."
}


In [ ]:
sample_2 = {
    'town': 'Corner brook',
    'identify_as': 'Female',
    'specify_below': ' ',
    "why_internship": "I have an interest in tech and hope to have a tech profession in the future. This opportunity will help me solidify that dream.",
    "tech_experience": "I am able to research online, use Google Sheets, and word for school work. I am also play games on PlayStation.",
    "non_tech_experience": "I am a dedicated volunteer in my community. I come from a larger size family where team work makes the dream work where I am the second eldest so a lot of responsibilities fall onto me. I have played on many all star soccer teams and school teams all throughout my time in school and I am personable and approachable and eager to learn.",
    "goals": "I will be attending MUN in September for engineering.",
    "other_comments": "I am excited for this opportunity."
}


In [22]:
import pandas as pd

diversity_keywords = {
    'woman': 1,
    'female': 1,
    'non binary': 1,
    'non-binary': 1,
    'genderfluid': 1,
    'agender': 1,
    '2slgbtq+': 1,
    'slgbtq': 1,
    'lgbtq': 1,
    'gender diverse': 1,
    'racialized minority': 1,
    'person disability': 1,
    'person with disability': 1,
    'disability': 1,
    'aboriginal': 1,
    'indigenous': 1,
    'newcomer canada': 1,
    'francophone': 1,
}

max_score = len(set(diversity_keywords))  # normalization factor

def calculate_individual_diversity_score(sample):
    text = sample.get('identify_as', '')
    if pd.isna(text) or text.strip() == '':
        return 0
    text = text.lower()
    score = 0
    matched = set()
    for key in diversity_keywords:
        if key in text and key not in matched:
            score += diversity_keywords[key]
            matched.add(key)
    return round(min(score / max_score, 1), 2)

sample_1 ={
        'town': 'Bay Roberts',
        'identify_as': 'Female',  # Make sure to include this as it was in training
        'why_internship': 'I am unsure which course should i take after my high school, i am confused between computer science and doctor, So i think this internship will help me to explore the horizons',
        'tech_experience': 'I have no tech experience right now, but would love to expand the knowledge',
        'non_tech_experience': 'President of school debate club',
        'goals': 'I am unsure',
        'other_comments': 'Very excited about this opportunity!'
    }
# Calculate diversity score
score = calculate_individual_diversity_score(sample_1)
print(f"Diversity Score for sample_1: {score}")

# Append score to sample
sample_1['score'] = score


Diversity Score for sample_1: 0.06


In [23]:
sample_1.pop('identify_as', None)
sample_1

{'town': 'Bay Roberts',
 'why_internship': 'I am unsure which course should i take after my high school, i am confused between computer science and doctor, So i think this internship will help me to explore the horizons',
 'tech_experience': 'I have no tech experience right now, but would love to expand the knowledge',
 'non_tech_experience': 'President of school debate club',
 'goals': 'I am unsure',
 'other_comments': 'Very excited about this opportunity!',
 'score': 0.06}

In [24]:
# test cell
fields_to_clean = [
    'town', 'identify_as', 'specify_below',
    'why_internship', 'tech_experience', 'non_tech_experience', 'goals', 'other_comments'
]

# Clean all selected fields in the sample
cleaned_sample = {
    key: clean_and_lemmatize(value) if key in fields_to_clean else value
    for key, value in sample_1.items()
}

# Combine all fields into one text string for the model
def combine_fields(entry):
    return " ".join([entry.get(col, "") for col in fields_to_clean])

test_samples = [combine_fields(cleaned_sample)]

# Tokenize
inputs = tokenizer(test_samples, return_tensors="pt", padding=True, truncation=True, max_length=256)

# Move inputs and model to the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Run model prediction
with torch.no_grad():
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    preds = torch.argmax(probs, dim=1)

# Define confidence threshold
CONFIDENCE_THRESHOLD = 0.60

# Display result with threshold-based selection
for i, raw_text in enumerate(test_samples):
    predicted_label = preds[i].item()
    confidence = probs[i][predicted_label].item()

    if predicted_label == 1 and confidence >= CONFIDENCE_THRESHOLD:
        confidence_label = "✅ SELECTED"
    else:
        confidence_label = "❌ NOT SELECTED"

    if predicted_label == 1:
      label = 'selected based on model'
    else:
      label= 'not selected based on model'
    print(sample_1)
    print(f"\n➡️ Prediction based on confidence threshold: {confidence_label} (Confidence: {confidence:.2f})\n")
    print(f"\n➡️ Prediction of Model: {label}")


{'town': 'Bay Roberts', 'why_internship': 'I am unsure which course should i take after my high school, i am confused between computer science and doctor, So i think this internship will help me to explore the horizons', 'tech_experience': 'I have no tech experience right now, but would love to expand the knowledge', 'non_tech_experience': 'President of school debate club', 'goals': 'I am unsure', 'other_comments': 'Very excited about this opportunity!', 'score': 0.06}

➡️ Prediction based on confidence threshold: ✅ SELECTED (Confidence: 0.98)


➡️ Prediction of Model: selected based on model
